In [14]:
import sys
from androguard.core.bytecodes import apk, dvm
from androguard.core.analysis import analysis
from androguard.core.bytecodes.apk import APK
import os
import re
sys.path.append("."+os.sep+'Log')
import logging as log

In [15]:
import androguard
print(androguard.__version__)

3.3.5


In [16]:

import keras
keras.__version__

'2.13.1'

In [17]:
import tensorflow as tf
print(tf.__version__)

2.13.0


In [18]:
features = list()

In [19]:
def getPermissions(Path):
    app = APK(Path)
    permissions = app.get_permissions()
    for i in permissions:
        features.append(i)

In [20]:
def getAPICalls(Path):
    app = APK(Path)
    app_dex = dvm.DalvikVMFormat(app.get_dex())
    app_x = analysis.Analysis(app_dex)
    classes = [cc.get_name() for cc in app_dex.get_classes()]
    for method in app_dex.get_methods():
        methodBlock = app_x.get_method(method)
        if method.get_code() == None:
            continue
        for i in methodBlock.get_basic_blocks().get():
            for ins in i.get_instructions():
                output = ins.get_output()
                match = re.search(r'(L[^;]*;)->([^\(]*)', output)
                if match and match.group(1) not in classes:
                    if match.group(2) == "<init>":
                        continue
                    api = match.group()
                    if api in features:
                        continue
                    else:
                        features.append(api)



In [21]:
path = "C:/Users/SRMAP/Desktop/FDP Code/a.envisionmobile.caa.apk"

In [22]:
getPermissions(path)
getAPICalls(path)

In [23]:
# print(features)

In [24]:
import numpy as np
import os
import logging
import sys
sys.path.append("."+os.sep+'GetFeature')
#import GetFeature
sys.path.append("."+os.sep+'Log')
#"from log import Log
import pandas as pd
import threading
import random


In [25]:
APIFile = "C:/Users/SRMAP/Desktop/FDP Code/Features/API.txt"
permissionFile = "C:/Users/SRMAP/Desktop/FDP Code/Features/permission.txt"
featurelistPath = "C:/Users/SRMAP/Desktop/FDP Code/Features/featureList.csv"
featurelist = list()

In [26]:
def featureFromFile(filePath):
		"""
		Get the features to be extracted from one feature file
		:param filePath:  file path
		:return: features list
		"""
		if not os.path.exists(filePath):
			#self.logger.error(filePath+"does not exist")
			print(filePath+"does not exist")
			return
		ls=list()
		with open(filePath,'r')as f:
			while(True):
				line=f.readline()
				if not line:break
				s=line.replace("\n","")
				s=s.replace("\r","")
				ls.append(s)
		return ls



In [27]:
def generateFeatureList():
		"""
		Get the features to be extracted from all features
		:param :
		:return: features list from all feature files
		"""
		ls1 = featureFromFile(APIFile)
		ls2 = featureFromFile(permissionFile)      
		featurelist = ls1+ls2

		frame = pd.DataFrame(featurelist)
		frame.to_csv(featurelistPath,header=False,index=False)

		return featurelist

In [28]:
featureList = generateFeatureList()

In [29]:
# featureList 

In [30]:
APKlist_T = list()
Matrix_test = np.zeros((0, len(featureList)), dtype="int")

In [31]:
apkFeature = np.zeros((1,featureList.__len__()), dtype=int)
try:
    for p in features:
        if p in featureList:
            i = featureList.index(p)
            apkFeature[0][i] = 1
except Exception as e:
    print("feature matrix extraction errors.")

In [32]:
# for p in features:
#     print(p)

In [33]:
def getFeaturefromAPK(features):
		apkFeature = np.zeros((1,featureList.__len__()), dtype=int)
		try:
			for p in features:
				if p in featureList:
					i = featureList.index(p)
					apkFeature[0][i] = 1
		except Exception as e:
			print("feature matrix extraction errors.")
		return apkFeature

In [34]:
features = getFeaturefromAPK(features)

In [35]:
features

array([[1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0,
        0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        0, 0, 0, 1]])

In [36]:
test = [[1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 1, 1]]

In [64]:
from keras.models import load_model
model = load_model('new_model.h5')

In [65]:
# # load saved model
# import pickle
# with open('model_mlp' , 'rb') as f:
#     clf = pickle.load(f)


In [66]:
y_hat_proba = model.predict(features)

1/1 [==============================] - 0s 53ms/step


In [67]:
y_hat_proba

array([[0.05304701, 0.946953  ]], dtype=float32)

In [36]:
import tensorflow as tf

# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path='model.tflite')

# Allocate tensors (this is required to set up the model for prediction)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare the input data (make sure it matches the expected shape)
input_data = np.array(test).astype(np.float32)

# Set input tensor
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run the model
interpreter.invoke()

# Get the output
output_data = interpreter.get_tensor(output_details[0]['index'])

# Print or process the output
print(output_data)


[[0.05898496 0.94101506]]
